In [12]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from pyquery import PyQuery as pq
from urllib.parse import quote
import pymysql
import re


# config
MONGO_URL='localhost'
MONGO_DB='taobao'
MONGO_COLLECTION='products'

KEYWORD='ipad'

MAX_PAGE=10
SERVICE_ARGS=['--load-images=false','--disk-cache=true']


# 
# chrome_options=webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# browser=webdriver.Chrome(chrome_options=chrome_options)

# wait=WebDriverWait(browser,10)
# client=pymongo.MongoClient(MONGO_URL)
# db=client[MONGO_DB]
browser=webdriver.Chrome()
wait=WebDriverWait(browser,10)
price_pattern=re.compile('¥.*?([0-9]+)')

def index_page(page):
    print('正在爬取第 ',page,' 页。')
    try:
        url='https://s.taobao.com/search?q='+quote(KEYWORD)
        browser.get(url)
        if page>1:
            input=wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'#mainsrp-pager div.form > input')))
            submit=wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'#mainsrp-pager div.form > span.btn.J_Submit')))
            input.clear()
            input.send_keys(page)
            submit.click()
        wait.until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'#mainsrp-pager li.item.active > span'),str(page)))
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.m-itemlist .items .item')))
        get_products()
    except TimeoutException:
        index_page(page)
        
def get_products():
    html=browser.page_source
    doc=pq(html)
    items=doc('#mainsrp-itemlist .items .item').items()
    for item in items:
        deal=0
        match_price=re.search('¥.*?(\d+.\d+)',item.find('.price').text(),re.S)
        match_deal=re.search('(\d+)',item.find('.deal-cnt').text(),re.S)
        if match_deal is not None:
            deal=int(match_deal.group(1))
            
    
        product={
            'id':item.find('.pic-link').attr('data-nid'),
#             'image':item.find('.pic .img').attr('data-src'),
            'title':item.find('.title').text(),
            'price':float(match_price.group(1)),
            'deal':deal,
            'shop':item.find('.shop').text(),
            'location':item.find('.location').text()
        }
        save_to_mysql(product)
        print(product)
        
def save_to_mysql(result):
    keys=','.join(result.keys())
    values=','.join(['%s']*len(result))
    sql='INSERT INTO t_taobao({keys}) VALUES({values})'.format(keys=keys,values=values)
    db=pymysql.connect(host='localhost',user='root',password='limu123456',port=3306,db='spiders',charset='utf8')
    cursor=db.cursor()

    try:
        cursor.execute(sql,tuple(result.values()))
        db.commit()
    except Exception:
        db.rollback()
        print('发生异常：',Exception)
    db.close()
    

        
def main():
    for i in range(1,3):
        index_page(i)
    browser.close()
    
if __name__ == '__main__':
    main()
    print('OK')
    



正在爬取第  1  页。
caching sha2: succeeded by fast path.
{'id': '566940629944', 'title': '赠电动牙刷【2年保修】Apple/苹果 iPad 2018款 9.7英寸平板电脑', 'price': 2299.0, 'deal': 5456, 'shop': '绿森数码官方旗舰店', 'location': '浙江 杭州'}
caching sha2: succeeded by fast path.
发生异常： <class 'Exception'>
{'id': '566940629944', 'title': '赠电动牙刷【2年保修】Apple/苹果\niPad\n2018款 9.7英寸平板电脑', 'price': 2299.0, 'deal': 5456, 'shop': '绿森数码官方旗舰店', 'location': '浙江 杭州'}
caching sha2: succeeded by fast path.
{'id': '547137374482', 'title': 'Apple/苹果\niPad\n9.7英寸平板电脑 2017款正品国行 WIFI 开发票', 'price': 1928.0, 'deal': 26894, 'shop': '苏宁易购官方旗舰店', 'location': '江苏 南京'}
caching sha2: succeeded by fast path.
{'id': '566534680696', 'title': '[12期分期]Apple/苹果\niPad\n2018款 9.7英寸wifi新款平板电脑128G', 'price': 2298.0, 'deal': 12004, 'shop': '卓辰数码旗舰店', 'location': '浙江 杭州'}
caching sha2: succeeded by fast path.
{'id': '557150061918', 'title': '汽车车载手机平板电脑车用\nipad\n用品后座支架车上多功能后排夹pad', 'price': 39.0, 'deal': 4074, 'shop': 'pzoz数码旗舰店', 'location': '湖北 武汉'}
caching sha2: suc

caching sha2: succeeded by fast path.
{'id': '552801603455', 'title': 'Apple/苹果 iPad Pro 10.5英寸 平板电脑 2018新款iPad苹果pro9.7', 'price': 3958.0, 'deal': 526, 'shop': '领域数码通讯', 'location': '广东 深圳'}
caching sha2: succeeded by fast path.
{'id': '41780562505', 'title': 'Apple/苹果 iPad 2018款iPad air3 平板电脑\nipad\n2018新款9.7', 'price': 1828.0, 'deal': 377, 'shop': 'ygsd', 'location': '北京'}
caching sha2: succeeded by fast path.
{'id': '565563160059', 'title': 'Apple/苹果\nipad\nmini2平板电脑 迷你2 wifi16G 32G 7.9英寸3G版', 'price': 1120.0, 'deal': 752, 'shop': '银夫人服装', 'location': '广东 东莞'}
caching sha2: succeeded by fast path.
{'id': '528654378962', 'title': '新款Apple/苹果\niPad\nPro 10.5寸Wifi 4G平板电脑 国行港版未激活', 'price': 4058.0, 'deal': 216, 'shop': '李小麦的店', 'location': '北京'}
caching sha2: succeeded by fast path.
{'id': '566919261059', 'title': '低至2268起/Apple/苹果\niPad\n2018款 9.7英寸wifi新款平板\nipad\nwifi新款平板电脑苹果 4G Cellular', 'price': 2278.0, 'deal': 996, 'shop': '环球机库官方旗舰店', 'location': '江苏 扬州'}
caching sha2: succeeded 

In [16]:
# 创建数据库spiders

import pymysql
db=pymysql.connect(host='localhost',user='root',password='limu123456',port=3306)
cursor=db.cursor()
cursor.execute('CREATE DATABASE spiders DEFAULT CHARACTER SET utf8 ')
db.close()

caching sha2: succeeded by fast path.


ProgrammingError: (1007, "Can't create database 'spiders'; database exists")

In [11]:
import pymysql
db=pymysql.connect(host='localhost',user='root',password='limu123456',port=3306,db='spiders')
cursor=db.cursor()
sql='CREATE TABLE t_taobao(id VARCHAR(255),title VARCHAR(255),price FLOAT,deal INT,shop VARCHAR(40),location CHAR(20),PRIMARY KEY(id))'
cursor.execute(sql)
db.close()

caching sha2: succeeded by fast path.
